# db2

In [1]:
from ipynb.fs.full.koselleck import *

In [2]:
PERIOD_LENS={5,20,70}
HDF_FORMAT='fixed'

## Model definitions

In [3]:
#FN_DB=os.path.join(PATH_DATA,'db3.koselleck2.h5')
FN_DB='/home/ryan/db2/db.koselleck3.h5'
# os.remove(FN_DB)
DBOBJ=None

In [4]:
def get_db(mode='a'):
    global DBOBJ
    if DBOBJ is not None: return DBOBJ
    DBOBJ=pd.HDFStore(
        FN_DB,
        mode=mode,
        complevel=9,
        complib='blosc'
    )
    return DBOBJ

In [5]:
# db=get_db()
# print(db['/vecs/bpo/1720-1740/1'])

In [6]:
def db_has_key(key):
    return key in get_db()
    with get_db(mode='r') as db:
        return key in db

In [7]:
db_has_key('test')

False

In [8]:
#with get_db() as db: db['test']=pd.Series(['Hello', 'world'])

## Vectors

In [9]:
dfmodels_all = get_pathdf_models(period_len=None)
# dfmodels_all

Scanning directory for models: 3901it [00:00, 68600.53it/s]


In [10]:
dfmodels=dfmodels_all[dfmodels_all.period_len.isin(PERIOD_LENS)]
# dfmodels

In [11]:
# %%timeit
# vwords=set(get_valid_words())

In [12]:
def do_hdf_ingest_vecs(row,only_valid_words=True):
    mpath=row.path
    m=load_model(mpath)
    data=m.wv.vectors
    keys=[m.wv.index_to_key[i] for i in range(len(data))]
    res=pd.DataFrame(data, index=keys)
    if only_valid_words:
        vwords=set(get_valid_words())
        res=res.loc[set(res.index) & vwords]
    qstr=f'/vecs/{row.corpus}/{row.period}/{int(row.run.split("_")[-1])}'
    return (qstr,res)

def hdf_ingest_vecs(dfmodels, only_valid_words=True):
    objs = [row for i,row in dfmodels.iterrows()]
    iterr=pmap_iter(do_hdf_ingest_vecs, objs, num_proc=1)
    for qstr,qdf in iterr:
        qdf.to_hdf(FN_DB,key=qstr)

In [13]:
def read_hdf(fn,key,**opts):
    try:
        return pd.read_hdf(fn,key,**opts)
    except KeyError:
        return pd.DataFrame()

In [24]:
# hdf_ingest_vecs(dfmodels)

In [27]:
def dbget(key):
    try:
        return get_db().get(key)
    except KeyError:
        return pd.DataFrame()

In [28]:
def vecs(period,run=1,corpus=DEFAULT_CORPUS):
    qstr=f'/vecs/{corpus}/{period}/{run}'
    #return get_db().get(qstr)
    #return read_hdf(FN_DB,qstr)
    return dbget(qstr)

In [31]:
# vecs('1820-1840',423432)

## Distances

In [32]:
def vecs2dist(dfvecs,only_valid_words=True,lim=None):
    if only_valid_words: dfvecs=dfvecs.loc[set(dfvecs.index)&set(get_valid_words())]
    if lim: dfvecs=dfvecs[:lim]
    dfsim=pd.DataFrame(
        fastdist.cosine_pairwise_distance(
            dfvecs.values.astype(float),
            return_matrix=True
        ),
        index=dfvecs.index,
        columns=dfvecs.index
    )
    dfdist=1-dfsim
    return dfdist

In [33]:
# vecs2dist(vecs(random.choice(periods)))

In [34]:
def ingest_dists_model(
        period,run=1,corpus=DEFAULT_CORPUS,
        only_valid_words=True,lim=25000,force=False,**attrs):
    dfvecs=vecs(period,run)
    dfdist=vecs2dist(dfvecs,only_valid_words=only_valid_words,lim=lim)
    return dfdist

def ingest_dists_model_(objd): return ingest_dists_model(**objd)

In [35]:
def ingest_dists(dfmodels,only_valid_words=True,lim=None,num_proc=1,force=False):
    dfmodels['run_int']=dfmodels.run.apply(lambda x: int(x.split('_')[-1]))
    objs = [
        dict(
            period=period,
            run=run,
            corpus=corpus,
            only_valid_words=only_valid_words,
            lim=lim,
            qstr=f'/dists/{corpus}/{period}/{run}'
        )
        for period,run,corpus in zip(dfmodels.period, dfmodels.run_int, dfmodels.corpus)
    ]
    if not force: objs = [d for d in objs if not db_has_key(d['qstr'])]
            
    iterr=pmap_iter(ingest_dists_model_, objs, num_proc=num_proc)
    #with get_db(mode='a') as db:
    for idx,odf in zip(objs, iterr):
        if odf is None: continue
        #odf.to_hdf(FN_DB, key=idx['qstr'])
        get_db().put(key=idx['qstr'], value=odf, format=HDF_FORMAT)

In [ ]:
ingest_dists(dfmodels,num_proc=1)

Mapping ingest_dists_model_() [x1]:   0%|          | 1/983 [00:16<4:31:01, 16.56s/it]